In [41]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from wikipediaapi import Wikipedia
import nltk
#Run nltk.download('punkt')


In [44]:
model = SentenceTransformer("all-mpnet-base-v2")

wiki = Wikipedia('RAGPractice', 'en')
doc = wiki.page("Taylor_Swift").text
doc = doc.split(',')


In [66]:
#API_KEY = Z1xwIDamyy1DcqXN1FN0vg75HCokwcMqGmSWhyqi
# https://api.data.gov/congress/v3?api_key=

import requests

api_token = '0b65b992cc93c03a21e1c82b88e1f6d6ea0e80e6'
# Define the endpoint and parameters
url = "https://www.courtlistener.com/api/rest/v3/opinions/"
params = {
    'search': 'disbarred',  # Replace with your keywords
    # 'court': 'scotus',  # Optional: Specify a court, e.g., Supreme Court of the United States
    'date_filed__gte': '2020-01-01',  # Optional: Filter by date
}

headers = {
    'Authorization': f'Token {api_token}'
}


# Make the request
response = requests.get(url, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Process the data
    for opinion in data['results']:
        print(opinion)
        # print(f"Case Name: {opinion['case_name']}")
        # print(f"Date Filed: {opinion['date_filed']}")
        # print(f"URL: {opinion['absolute_url']}")
        # print()
else:
    print("Failed to retrieve data:", response.status_code)

{'resource_uri': 'https://www.courtlistener.com/api/rest/v3/opinions/10792057/', 'id': 10792057, 'absolute_url': '/opinion/10325469/saul-vega-v-joseph-canestraro/', 'cluster_id': 10325469, 'cluster': 'https://www.courtlistener.com/api/rest/v3/clusters/10325469/', 'author_id': None, 'author': None, 'joined_by': [], 'date_created': '2025-02-02T13:00:35.293876-08:00', 'date_modified': '2025-02-02T13:44:58.021294-08:00', 'author_str': '', 'per_curiam': False, 'joined_by_str': '', 'type': '010combined', 'sha1': 'af7d3d2ea372cbe95bcd7898391a40cc692f5d19', 'page_count': 2, 'download_url': 'https://www.govinfo.gov/content/pkg/USCOURTS-ca4-24-06723/pdf/USCOURTS-ca4-24-06723-0.pdf', 'local_path': 'pdf/2025/01/31/saul_vega_v._joseph_canestraro.pdf', 'plain_text': 'USCA4 Appeal: 24-6723      Doc: 17         Filed: 01/31/2025     Pg: 1 of 2\n\n\n\n\n                                             UNPUBLISHED\n\n                               UNITED STATES COURT OF APPEALS\n                            

In [59]:
encoded_doc = model.encode(doc, convert_to_tensor=True)

query = "What is taylor swift's first album?"
encoded_query = model.encode(query, convert_to_tnesor=True)

hits = util.semantic_search(encoded_query, encoded_doc, top_k=3)
print(hits)
hits = hits[0]



[[{'corpus_id': 211, 'score': 0.7098111510276794}, {'corpus_id': 62, 'score': 0.7055413126945496}, {'corpus_id': 2, 'score': 0.6893085837364197}]]


In [61]:
context = ''
for i, hit in enumerate(hits):
    context += doc[hit['corpus_id']]
    print(f"Hit {i}:", doc[hit['corpus_id']])

print(context)

Hit 0:  A photobook, The Official Taylor Swift: The Eras Tour Book, was released on November 29, 2024, and sold over a million copies in its first week in the US alone
Hit 1:  Swift released two EPs, The Taylor Swift Holiday Collection in October 2007 and Beautiful Eyes in July 2008
Hit 2: 
Swift signed to Big Machine Records in 2005, debuting as a country singer with the albums Taylor Swift (2006) and Fearless (2008)
 A photobook, The Official Taylor Swift: The Eras Tour Book, was released on November 29, 2024, and sold over a million copies in its first week in the US alone Swift released two EPs, The Taylor Swift Holiday Collection in October 2007 and Beautiful Eyes in July 2008
Swift signed to Big Machine Records in 2005, debuting as a country singer with the albums Taylor Swift (2006) and Fearless (2008)


In [62]:
prompt = f'Answer the provided query with the provided context without adding additional information outside the context. CONTEXT: {context} QUERY: {query}'

print(prompt)

Answer the provided query with the provided context without adding additional information outside the context. CONTEXT:  A photobook, The Official Taylor Swift: The Eras Tour Book, was released on November 29, 2024, and sold over a million copies in its first week in the US alone Swift released two EPs, The Taylor Swift Holiday Collection in October 2007 and Beautiful Eyes in July 2008
Swift signed to Big Machine Records in 2005, debuting as a country singer with the albums Taylor Swift (2006) and Fearless (2008) QUERY: What is taylor swift's first album?


In [63]:
from ollama import chat, ChatResponse

response: ChatResponse = chat(model='llama3.2', messages= [
                              {
                                  'role':'user',
                                  'content': prompt,
                                  'stream': True
                              }])

print(response['message']['content'])

Taylor Swift's first album was "Taylor Swift" released in 2006.
